Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# CSE204 - Introduction to Machine Learning - Lab Session 7: Building a Neural Network From Scratch - Backpropagation

<img src="https://raw.githubusercontent.com/adimajo/CSE204-2021/master/data/logo.jpg" style="float: left; width: 15%" />

[CSE204-2021](https://moodle.polytechnique.fr/course/view.php?id=12838) Lab session #07

J.B. Scoggins - Adrien Ehrhardt

This lab is a direct follow-up of Lab session #06 on Neural networks. We will now make the network trainable and... train it!

### Recall last lab's notations

We will consider simple feed-forward networks that can be described by the following recursive relationship,

$$
\begin{align}
&z^l = a^{l-1} W^l + b^l,\\
&a^l = \sigma^l(z^l),
\end{align}
$$

where $a^l$ is the output (activation) of layer $l$ which is a nonlinear function $\sigma^l$ of a linear transformation of the previous layer's output.  The linear transformation is performed using the weight matrix $W^l$ and bias vector $b^l$ associated with the layer $l$.  We will denote the last layer in the network with a capital $L$ superscript.  The recursion is stopped by setting $a^0 = x$, where $x$ is the input vector to our network.  Note that in the recursive expressions above, we implicitly assume that our input/output vectors are row vectors.  The reason for this will be evident later.

Taking this notation into account, we see that a network with $L-1$ hidden layers is fully expressed by its $L$ weight matrices, bias vectors, and activation functions.  We can denote the set of trainable parameters in our network by $\theta = \{ W^1, \dots, W^L, b^1, \dots, b^L \}$.

In this lab, we are only concerned with supervised learning tasks.  Recall that in supervised learning, we have a dataset represented by a list of $(x, y)$ pairs where $x$ is the input to our model and $y$ is the desired output.

- For regression problems where we want to fit a function $y = f(x)$, $x$ is the independent variable vector, and $y$ is the function value.
- In classification problems, $x$ will correspond to a set of attributes and $y$ the corresponding label.

The goal of supervised learning is to "train" our network by adjusting its parameters in order to minimize a cost function over the entire training set,

$$
\min_\theta \mathcal{L} = \min_\theta \sum_{i=1}^N \ell^{(i)}
$$

where $\ell^{(i)} = \ell(\sigma^L(\sigma^{L-1}(\dots(x^{(i)}W^1 + b^1)\dots)W^L + b^L), y^{(i)})$ and $\ell(\hat{y}, y)$ denotes the particular form of the loss function being considered.  In this lab, we will use 2 different loss functions:

1. Quadratic Loss: $\ell(\hat{y}, y) = \|\hat{y} - y\|^2$
2. Cross-entropy Loss: $\ell(\hat{y}, y) = -[ y \ln\hat{y} + (1-y)\ln(1-\hat{y})]$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# This might take some time and output warnings: don't worry.
# This might error, e.g. if you didn't install the CSE204 kernel.
# You can try !pip install tensorflow in a new cell to install it on the fly
# but be sure the remove the direcctive before uploading your work!
import tensorflow as tf

Here are the activations classes you (hopefully) managed to implement last week:

In [ ]:
class Activation(object):
    """
    Base activation node class
    """

    def __new__(self, x: float) -> float:
        """
        What to do when instantiating an object: this basically says Activation(x)
        will be equivalent to Activation()(x).

        :param float or numpy.array x: input of the activation
        :return: call the activation function
        :rtype: float or numpy.array
        """
        return self.__call__(x)

In [ ]:
class Identity(Activation):
    """
    Identity activation node
    """

    @staticmethod
    def __call__(x: float) -> float:
        """
        Implements the identity activation function

        :param float or np.array x: input of the activation
        :return: identity of input
        :rtype: float or np.array
        """
        return x

    @staticmethod
    def prime(x: float) -> float:
        """
        Implements the derivative of the identity activation function

        :param float or numpy.array: input of the activation
        :return: derivative of input
        :rtype: float or numpy.array
        """
        return np.ones_like(x)

In [ ]:
class Threshold(Activation):
    """
    Threshold activation node
    """

    @staticmethod
    def __call__(x: float) -> float:
        """
        Implements the threshold activation function

        :param float or numpy.array x: input of the activation
        :return: if x > 0, 1, else 0
        :rtype: float or numpy.array
        """
        return np.where(x > 0.0, 1.0, 0.0)

    @staticmethod
    def prime(x: float) -> float:
        """
        Implements the derivative of the threshold activation function

        :param float or numpy.array: input of the activation
        :return: derivative of input
        :rtype: float or numpy.array
        """
        return np.zeros_like(x)

In [ ]:
class Sigmoid(Activation):
    """
    Sigmoid activation node
    """
    @staticmethod
    def __call__(x: float) -> float:
        """
        Implements the sigmoid activation function

        :param float or numpy.array x: input of the activation
        :return: sigmoid transform of input
        :rtype: float or numpy.array
        """
        return 1.0 / (1.0 + np.exp(-x))

    @staticmethod
    def prime(x: float) -> float:
        """
        Implements the derivative of the sigmoid activation function
        (you can make good use of Sigmoid(...)!)

        :param float or numpy.array: input of the activation
        :return: derivative of input
        :rtype: float or numpy.array
        """
        sigmoid = Sigmoid(x)
        return sigmoid * (1.0 - sigmoid)

Here is the `Network` class you (hopefully) managed to implement last week.
**Notice that the implementation and signature of the `feed_forward` method changed so as not to recompute things in `back_propagation`** (see Question 1.3).

A `loss` attribute has also been added to record the evolution of the loss.

In [ ]:
class Network:
    """
    A simple implementation of a feed-forward artificial neural network.
    Work on this class throughout the entire exercise, rerunning this cell after each update.
    """

    def __init__(self, sizes: list, activations: list):
        """
        Construct a network given a list of the number of neurons in each layer and
        a list of activations which will be applied to each layer.

        :param list sizes: A list of integers representing the number of nodes in each layer,
        including the input and output layers.
        :param list activations: A list of callable objects representing the activation functions.
        Its size should be one less than sizes.
        """
        self.sizes = sizes
        self.sigmas = activations
        self.loss = []

        self.biases = [np.random.randn(1, n) for n in sizes[1:]]
        self.weights = [
            np.random.randn(m, n) for m, n in zip(sizes[:-1], sizes[1:])
        ]
        # Optional: better initialization
        # self.weights = [
        #     np.random.randn(m, n) / np.sqrt(m) for m, n in zip(sizes[:-1], sizes[1:])
        # ]

    def num_params(self) -> int:
        """Returns the total number of trainable parameters in this network."""
        return sum([m * n + n for m, n in zip(self.sizes[:-1], self.sizes[1:])])

    def feed_forward(self, x: np.array):
        """
        Evaluates the network for the given input and returns the intermediary results.

        :param numpy.array x: A numpy 2D-array where the columns represent input variables
        and rows represent independent samples.
        :return: output of the network
        :rtype: np.array, list, list
        """
        activations = [x]
        zs = []

        # Forward pass, storing z and a as we go
        for W, b, sigma in zip(self.weights, self.biases, self.sigmas):
            zs.append(np.matmul(activations[-1], W) + b)
            activations.append(sigma(zs[-1]))
        return activations[-1], activations, zs

    def back_propagation(self, x: np.array, y: np.array, cost, verbose: bool = False):
        """
        Compute gradients of cost function w.r.t network parameters for a single training point.

        :param numpy.array x: a single input in a 2D array (1, #features)
        :param numpy.array y: a single output
        :param Cost cost: a cost class
        :param bool verbose: whether to print debugging information (you don't have to implement this feature)
        :return: list of gradients of weights, list of gradients of biases
        :rtype: list, list
        """
        # Exercise 1.3
        # YOUR CODE HERE
        raise NotImplementedError()
        return w_grad, b_grad

    def update_step(self, x_train: np.array, y_train: np.array, cost, learning_rate: float,
                    verbose: bool = False):
        """
        Compute the average parameter gradients over the whole training set and do a GD step.
        N.B.: this does not return anything!

        :param numpy.array x_train: A design matrix (2D array (#samples, #features))
        :param numpy.array y_train: A vector / array of responses
        :param Cost cost: A callable object with signature cost(y_pred, y) for computing the cost of a single
        training example.
        :param float learning_rate: The learning rate to use in the GD step.
        :param bool update_step: Perform an update step: average all the gradients and update the parameters.
        """
        # Storage room for sum of training samples gradients
        w_grad_sum = [np.zeros_like(w) for w in self.weights]
        b_grad_sum = [np.zeros_like(b) for b in self.biases]
        # Exercise 1.4
        ## Suggestions
        # Compute gradients of weights and biases for all layers and samples
        # Sum gradients for all samples
        # Update the weights and biases
        # YOUR CODE HERE
        raise NotImplementedError()

    def train(self, x_train, y_train, cost, learning_rate: float, epochs: int = 100, verbose: bool = True):
        """
        Trains this network on the training data, i.e. iterate the update step.
        N.B.: this does not return anything!

        :param np.array x_train: A design matrix
        :param np.array y_train: A vector of responses
        :param Cost cost: A callable object with signature cost(y_pred, y) for computing the cost of a single
        training example.
        :param float learning_rate: The learning rate to use in the GD step.
        :param int epochs: The number of epochs to train with.
        :param bool verbose: Whether to print the loss at each epoch.
        """
        for i in range(epochs):
            self.update_step(x_train, y_train, cost, learning_rate)

            # Compute the loss
            y_pred, _, _ = self.feed_forward(x_train)
            loss = cost(y_pred, y_train)
            self.loss.append(loss)
            if verbose:
                print('epoch:', i, '\n\tloss: ', loss)

### Exercise 1: Implement the training algorithm

Before we can implement the training algorithm, we need to define a cost function. For now, let's create a base class `Cost`, similar to the base class `Activation`, for simplicity, and a `QuadraticCost` class representing the quadratic cost which was given in the introduction.

In [ ]:
class Cost:
    def __new__(self, y_pred: np.array, y: np.array) -> float:
        return self.__call__(y_pred, y)

1. Implement the quadratic cost defined above in the function below.

In [ ]:
class QuadraticCost(Cost):
    @staticmethod
    def __call__(y_pred: np.array, y: np.array) -> float:
        """
        Computes the quadratic cost function.

        :param numpy.array y_pred: y_hat for each sample
        :param numpy.array y: y for each sample
        :return: Squared L2 norm of the difference
        :rtype: float
        """
        # YOUR CODE HERE
        raise NotImplementedError()
    
    @staticmethod
    def derivative(y_pred: np.array, y: np.array) -> np.array:
        """
        Computes the derivative of the quadratic cost function.

        :param numpy.array y_pred: y_hat for each sample
        :param numpy.array y: y for each sample
        :return: derivative of cost function w.r.t. y_pred
        :rtype: numpy.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
assert QuadraticCost(0, 0) == 0
assert QuadraticCost.derivative(0, 0) == 0
assert QuadraticCost(np.array([0, 0]), np.array([0, 0])) == 0

2. Implement the `back_propagation` function in the `Network` class.

With this cost defined, let's implement three remaining functions in `Network` that will allow us to train our models. The first is the `back_propagation` method which takes a single training sample $x, y$ and a `Cost` class  like the one above, and computes the gradient of the cost function w.r.t. to all the weights and biases using the backpropagation algorithm. Use the formulas found [at the end of lab #06](https://adimajo.github.io/CSE204-2021/lab_session_06/lab_session_06.html). It sould return the lists of $\nabla_{W^l}\ell_p$ and $\nabla_{b^l}\ell_p$ for $1 \leq l \leq L$.

**Remember to execute the `Network` class again before running the tests below.**

In [ ]:
# Define a single training point
x = np.array([1, 2]).reshape(1, 2)
y = np.array([0])

# Define a network
network = Network([2, 5, 1], [Sigmoid, Identity])

# Perform back-propagation
w_grad, b_grad = network.back_propagation(x, y, QuadraticCost)

assert len(w_grad) == 2  # 2 layers (1 hidden, 1 output)
assert len(b_grad) == 2  # 2 layers (1 hidden, 1 output)

3. Implement the `update_step` function in the `Network` class.

Next, implement a single epoch of training by **averaging** the gradients over the whole dataset (calling `back_propagation` on each data point) and using the GD update rule to update the weights and biases.

The `update_step` method is used by the `train` method successively in a loop as you can see in the `Network` class implementation above.  Assuming you have correctly implemented the `backprop` and `update_step` functions correctly, you should now be able to train the network on real training data.

**Remember to execute the `Network` class again before running the tests below.**

In [ ]:
# Define a single training point
x = np.random.uniform(-1, 1, (100, 1))
y = np.sin(np.pi * x)

# Define a network
network = Network([1, 100, 1], [Sigmoid, Identity])

# Store weights and biases before the update to see if something changed
weights_before = network.weights[0].copy()
biases_before = network.biases[0].copy()


# Perform an update step
network.update_step(x, y, QuadraticCost, 0.1)

# Assert that something did change
assert np.any(np.not_equal(network.weights[0], weights_before))
assert np.any(np.not_equal(network.biases[0], biases_before))


### Exercise 2: Test training on simple regression problem.

Run the following code to test the network.  If all is correct, you should see the total loss dropping and the network will learn the sine function.  

1. Play with different hyperparameters of the network and training such as the number of layers, number of nodes per layer, training rate, and number of training samples. Can you improve the training performance?

In [ ]:
# Hyperparameters to tune
n_iterations = 10000
lr = 0.15
verbose = False
sizes = [1, 3, 3, 1]
activations = [Sigmoid, Sigmoid, Identity]

In [ ]:
# Training the network
np.random.seed(1)
network = Network(sizes, activations)

network.train(x_train=x, y_train=y, cost=QuadraticCost,
              learning_rate=lr, epochs=n_iterations, verbose=verbose)

In [ ]:
# Some test samples
x_pred = np.arange(-1, 1, 0.01).reshape(200, 1)
y_pred, _, _ = network.feed_forward(x_pred)

In [ ]:
# Plot train samples, truth, test samples and predictions
plt.plot(x, y, 'k.', label="Truth")
plt.plot(x_pred, y_pred, 'k-', label="Prediction")
plt.legend()
plt.show()
plt.plot(range(n_iterations), network.loss, label="Training loss")
plt.legend()
plt.show()
plt.plot(range(int(n_iterations / 2), n_iterations),
         network.loss[int(n_iterations / 2):],
         label=f"Training loss, last {int(n_iterations / 2)} observations")
plt.legend()
plt.show()

YOUR ANSWER HERE

## Step 3: Fashion items classification

Let's test our model on a more sophisticated dataset, namely [Zalando research's fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). This dataset includes 70,000 images of fashion items 28x28 pixels (=784), split into a training set of 60,000 images and a test set of 10,000 images. Each image is labeled with a digit from 0 to 9.  

The original [MNIST dataset](http://yann.lecun.com/exdb/mnist/) consists in handwritten digits, e.g. to automatically read the addresses on mails. The performance of neural networks on this task was so much better than previous approaches that it (re-)sparked a lot of interest in them (and brought fame to Yann LeCun). It is now considered an "easy" task.

### Exercise 3: Get to know the dataset

1. Just run the following cell to download the MNIST database.

In [ ]:
# This might take some time to download the datasets.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

2. Next check that the shapes of the training and test data, and their content, are what you expect.

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

YOUR ANSWER HERE

3. Get a feel for the dataset by displaying some images.

*Python hint*: [plt.imshow](https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.imshow.html).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Exercise 4: Create a one-hot encoding of the labels

`y_train` and `y_test` currently contain the digits 0-9, which is not suited for training as we want to perform a 10-class classification training on our network. To fix this, we need to perform a so-called one-hot encoding on the labels which convert each digit into a vector of length 10 full of zeros, except the element corresponding to the digit which should be 1.  For example, if our label is 3, then the corresponding one-hot encoding is [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].

1. Convert the `y_train` and `y_test` vectors into one-hot encodings.

In [ ]:
# y_train =  ...
# y_test =  ...
# YOUR CODE HERE
raise NotImplementedError()

### Exercise 5: Train your network to recognize fashion items

1. Implement the cross-entropy loss function below.

In [ ]:
class CrossEntropy(Cost):
    """Cross Entropy loss function"""
    @staticmethod
    def __call__(y_pred: np.array, y: np.array) -> float:
        """
        Value of the cross entropy loss given predictions and true labels

        :param numpy.array y_pred: 1D array associated with the predicted probability of y=1
        :param numpy.array y: 1D array of true (0/1) classes
        :return: value of the loss
        :rtype: float or numpy.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()

    @staticmethod
    def derivative(y_pred: np.array, y: np.array) -> np.array:
        """
        Derivative of the cross entropy loss function

        :param numpy.array y_pred: 1D array associated with the predicted probability of y=1
        :param numpy.array y: 1D array of true (0/1) classes
        :return: gradient of the loss w.r.t. y_pred
        :rtype: numpy.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
assert CrossEntropy(0.5, 0) == - np.log(0.5)
assert CrossEntropy.derivative(0.5, 0) == 2.0
assert int(CrossEntropy(np.array([0.5, 0.5]),
                    np.array([0, 1]))) == 1
assert CrossEntropy.derivative(0.5, 0) == 2.0

2. Instanciate a network with 1 hidden layer of 30 nodes, with sigmoid activations and a cross-entropy loss.

In [ ]:
# network = ...
# YOUR CODE HERE
raise NotImplementedError()

3. Train the network using the training set, any learning rate / number of iterations (see 4.).

In [ ]:
# Train the network for X epochs with a learning rate of Y
# Our implementation does not deal with 3D inputs, so you need
# to "flatten" the 28x28 images in a single vector / observation / sample / row.
# YOUR CODE HERE
raise NotImplementedError()

4. Plot the evolution of the loss and go back to training, tweaking the learning rate / number of iterations until the plot looks satisfactory.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

3. Compute the accuracy of the model on the test data and compare a few predictions to their images (see labels [here](https://github.com/zalandoresearch/fashion-mnist#labels)).

In [ ]:
# Compute the accuracy
# accuracy = ...

# For example, display an example of a class probability and its true value
# YOUR CODE HERE
raise NotImplementedError()

## Extra

At this point you have a fully functioning neural network model which can be used in real machine-learning problems.  There are many things which you could add at this point to make the model more effective.  Consider playing with some of the following:

- Adding weight regularization
- Using different activation functions such as ReLU or Tanh
- Improve the efficiency by making the backprop algorithm work on whole batches at once
- Implement the SGD algorithm
- Redesign the model to allow plug-and-play layers to be stacked together, allowing more complicated layers such as convolutions to be used.

Of course the list is endless, just have fun!

## Next Step: Using Tensorflow

As you have now seen, building a neural network and training from scratch can require a significant amount of effort.  Furthermore, the naive design of our `Network` class does not allow us to easily incorporate new kinds of layers such as convolutions, dropout, or batch normalization.  In practice of course, several libraries already exist that provide all of this functionality for you, so that you can focus on building a model for your specific dataset.  Here, we will show you how to redo the digit classification example above, with less than 10 lines of code using the Tensorflow library. You can [find this example](https://www.tensorflow.org/tutorials) directly on the Tensorflow webiste if you are interested.  Let's walk through the code below.

### Import Tensorflow and download the MNIST dataset

Tensorflow includes a high-level neural network architecture building API called Keras which makes it easy to create neural network models using "off-the-shelf" layers.  In addition, Keras also provides commonly used datasets, such as the MNIST dataset.  In the first line below, Tensorflow is imported.  Second, the `keras.datasets` API is used to dowload the full dataset into two groups, a training set and a testing set, like you have seen before.  In the third line, the pixel data (integers between 0-255) is converted to real values between 0.0-1.0, making it easier for our NN model to learn.

### Build the NN model

In the next two lines, the Keras API is used to create a simple NN model with the following layers:

- `Flatten` takes an N-dimensional array and reshapes it into a 1D array. In this case, the images are 28x28 pixels, which get converted to a 784 1D tensor.
- `Dense` is exactly like the type of implicit layers we coded in our `Network` class.  That is, they take some 1D input and apply a linear transformation before applying a nonlinear activation.  In this case we are using 512 nodes in the dense layer.
- `Dropout` is a little more advanced.  As you have seen in the lecture, a dropout layer sets some percentage of the inputs to the layer to zero.  During training, each time data is propagated through the dropout layer, a different random set of nodes is zeroed out.  This forces the network to make more generalizations about the data and reduces overfitting.  When training is done, the dropout layer allows all the nodes to pass through, but multiplies the result by the dropout factor such that the sum remains the same.  In this case, 20% of the nodes are randomly canceled out during training and the result during evaluation is multiplied by 0.8.  
- The final layer is a dense layer of 10 nodes corresponding to the one-hot encoding of the digit labels.  Note that a softmax activation function is used so that these nodes will represent propbabilities.

In the last line, the model is "compiled" with an optimizer (Adam), loss function (sparse categorical crossentropy), and a metrics, which is responsible for providing useful output during training.  In this case, the `accuracy` metric is used which computes the total number of training samples that were correctly categorized as trianing progresses.

You may modify the model as you wish.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train and evaluate the model

In the last step, we train the model.  This happens in the first line below, were we call the model's `fit` function, which takes the training data and a number of epochs.  The number of batches per epoch is handled by default.  In the second line, we evaluate the model on our testing data to see how well it does on data it has never seen before.  Note that we get a ~ 79% accuracy with this simple model!

In [ ]:
n_epoch = 5

In [ ]:
model.fit(x_train, y_train, epochs=n_epoch)
loss, acc = model.evaluate(x_test, y_test)
print("Final test loss:", loss)
print("Final test accuracy:", acc)

In [ ]:
print("The probabilities for each class for the first test point:\n", model.predict(x_test)[0, :].reshape(-1, 1))
print("The true class of the first test point:\n", y_test[0].reshape(-1, 1))
plt.imshow(x_test[0, :]);

Let's do the same with PyTorch. There are many deep learning frameworks, but Tensorflow and PyTorch are the most used, with Tensorflow becoming out-of-fashion (!) in favor of PyTorch, generally dubbed "simpler" (well, we'll see how that turns out!).

(Tensorflow is indeed more complicated but in the code above, we relied on Keras, a library built on top and meant to simplify the usage of several deep learning frameworks.)

We did **not** install PyTorch with the CSE204 kernel. So proceed by installing PyTorch if you wish to play with it (what follows is not graded).

In [ ]:
try:
    from torch import nn, optim, no_grad
except ModuleNotFoundError:
    !pip install torch torchvision
    from torch import nn, optim, no_grad

In [ ]:
from torch import float as torch_float
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

Re-download the MNIST dataset with the torch API / format (may take a while).

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Create data loaders (iterators over the training and testing data, think `range`).

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

Create the model as a class inheriting from nn.Module.
You must at least define `__init__`, where you might want to define the structure of your network, as well as the `forward` method (which is basically `feed_forward`).

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Instanciate your model by instanciating an object of this new class. You still need a loss function (our `Cost`), and an optimizer (we hard-coded Gradient Descent in `Network`).

Then we iterate on the training data for a given number of epochs, using a `for` loop. At each iteration, we must:
* Perform the feedforward pass;
* Compute the loss ($\delta^L$);
* Perform backpropagation by:
    * Calculating the loss and the gradients of $W$ and $b$ for $l < L$;
    * Update the parameters $W$ and $b$.

In [ ]:
model = NeuralNetwork()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, eps=1e-07)
model.train()  # Switch to training mode

for _ in range(n_epoch):
    for batch, (X, y) in enumerate(train_dataloader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

To compute the testing loss / error, we use the iterator over the test dataset and the loss function, which we previously defined.

In [ ]:
model.eval()  # Switch to evaluation mode
test_loss, correct = 0, 0
# Do not compute the gradient
with no_grad():
    for X, y in test_dataloader:
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch_float).sum().item()
test_loss /= len(test_dataloader.dataset)
correct /= len(test_dataloader.dataset)
print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Your turn (Bonus - 2 pts)
Play with tensorflow / keras and / or pytorch and provide your own model and training routine.

Try to separate the data into train / dev / test where train is used to train the model, dev is used to evaluate the model's performance at each epoch, and test is hold out until the end of the training.

You may want to test on other datasets. See [keras datasets](https://keras.io/api/datasets/) and [PyTorch datasets](https://pytorch.org/vision/stable/datasets.html).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()